In [2]:
using Random
using JSON3

In [3]:
function get_weights(x, max_interval)
    intervals = rand(0:max_interval, x-1)
    push!(intervals,0)
    push!(intervals,max_interval)
    intervals = sort(intervals)
    return diff(intervals)
end

function get_perm(n)
    P = Int16.(zeros(n,n))
    perm = randperm(n)
    for i=1:length(perm)
        P[perm[i], i] = 1
    end
    return P, perm
end

get_perm (generic function with 1 method)

In [4]:
n = 3
num_permutations = n

dataset_size = 10000
dataset = Dict([])
max_interval = 10^(length(string(n*n)) + 2)

for i in range(0,dataset_size)
  
  X = zeros(n,n)
  permutations = zeros(n,num_permutations)
  weights = get_weights(num_permutations, max_interval)

  # generate doubly stochastic matrix
  for j=1:num_permutations
    P, perm = get_perm(n)
    X = X .+ weights[j]*P
    permutations[:,j] = perm
  end
  # save doubly stochastic matrix, weights, and permutations
  entry = Dict([("permutations", Int16.(permutations)), 
    ("weights", weights./max_interval), 
    ("doubly_stochastic_matrix", X./max_interval),
    ("n",n)]
    )
  dataset[i] = entry

end

file_name = "data/dataset_" * string(n) * "_" * string(dataset_size) * "_sparse.json"

open(file_name,"w") do io
  JSON3.pretty(io, dataset)
end

### Recover DSM from permutations and weights

In [5]:
using JSON3
n = 3
data = JSON3.read("data/dataset_" * string(n) * "_10000_sparse.json")
D = data["1"].doubly_stochastic_matrix
D = reshape(D,n,n)

num_perm = n

M = Int16.(zeros(n*n,num_perm))
perm = Int16.(reshape(data["1"].permutations, n, num_perm))
for i=1:num_perm
    for j=1:n
        M[(j-1)*n + perm[j,i],i] = 1
    end
end

In [6]:
reshape(M*Float64.(data["1"].weights), n, n)

3×3 Matrix{Float64}:
 0.0    0.18   0.82
 0.839  0.161  0.0
 0.161  0.659  0.18

In [7]:
reshape(data["1"].doubly_stochastic_matrix, n, n)

3×3 reshape(::JSON3.Array{Union{Float64, Int64}, Vector{UInt8}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}}, 3, 3) with eltype Union{Float64, Int64}:
 0      0.18   0.82
 0.839  0.161  0
 0.161  0.659  0.18